In [161]:
!pip install gurobipy

In [162]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

In [163]:
capacity = pd.read_excel("/content/capacity.xlsx")
Demand = pd.read_excel("/content/demand.xlsx")
fixed_cost = pd.read_excel("/content/fixed_cost.xlsx")
Variable_cost = pd.read_excel("/content/variable_costs.xlsx")
freight_cost = pd.read_excel("/content/freight_costs.xlsx")

In [164]:
countries = capacity["Capacity (kUnits/month)"].tolist()
countries

['USA', 'Germany', 'Japan', 'Brazil', 'India']

In [165]:
capacity[["Low", "High"]] *= 1000
capacity.rename(columns={"Capacity (kUnits/month)": "Units/month"}, inplace=True)
capacity

,Units/month,Low,High
0,USA,500000,1500000
1,Germany,500000,1500000
2,Japan,500000,1500000
3,Brazil,500000,1500000
4,India,500000,3000000


In [166]:
Demand.columns = Demand.iloc[0]
Demand = Demand[1:].reset_index(drop=True)
Demand = Demand.iloc[:, 1:]
Demand

,(Units/month),Demand
0,USA,2800000
1,Germany,90000
2,Japan,1700000
3,Brazil,145000
4,India,160000


In [167]:
fixed_cost.rename(columns={"Unnamed: 0": "$/month"}, inplace=True)
fixed_cost

,$/month,Low,High
0,USA,6500,9500
1,Germany,4980,7270
2,Japan,6230,9100
3,Brazil,3230,4730
4,India,2110,6160


In [168]:
Variable_cost

,Variable Costs ($/Unit),USA,Germany,Japan,Brazil,India
0,USA,12,12,12,12,12
1,Germany,13,13,13,13,13
2,Japan,10,10,10,10,10
3,Brazil,8,8,8,8,8
4,India,5,5,5,5,5


In [169]:
freight_cost[countries] /= 1000
freight_cost.rename(columns={"Freight Costs ($/Container)": "Freight Costs ($/unit)"}, inplace=True)
freight_cost

,Freight Costs ($/unit),USA,Germany,Japan,Brazil,India
0,USA,0.000,12.25,1.100,16.100,8.778
1,Germany,13.335,0.00,8.617,20.244,10.073
2,Japan,15.400,22.75,0.000,43.610,14.350
3,Brazil,16.450,22.05,28.000,0.000,29.750
4,India,13.650,15.40,24.500,29.400,0.000


In [170]:
capacity_low = dict(zip(capacity["Units/month"], capacity["Low"]))
capacity_high = dict(zip(capacity["Units/month"], capacity["High"]))
demand = dict(zip(Demand["(Units/month)"], Demand["Demand"]))
freight_cost = freight_cost.set_index("Freight Costs ($/unit)").T.to_dict()
variable_cost = dict(zip(Variable_cost["Variable Costs ($/Unit)"], Variable_cost.iloc[:, 1]))
fixed_cost_low = dict(zip(fixed_cost["$/month"], fixed_cost["Low"]))
fixed_cost_high = dict(zip(fixed_cost["$/month"], fixed_cost["High"]))

In [171]:
# Initialize the optimization model
model = gp.Model("Minimize_Cost")

In [172]:
# Decision variables: Number of Low and High Capacity plants to open in each country
x_low = model.addVars(countries, vtype=GRB.INTEGER, name="x_low")
x_high = model.addVars(countries, vtype=GRB.INTEGER, name="x_high")

In [173]:
# Decision variables: Quantity of goods produced and distributed
y = model.addVars(countries, countries, vtype=GRB.CONTINUOUS, name="y")

In [174]:
# Objective function: Total cost = Fixed Cost + Variable Cost + Freight Cost
model.setObjective(
    gp.quicksum(fixed_cost_low[i] * x_low[i] + fixed_cost_high[i] * x_high[i] for i in countries) +
    gp.quicksum((variable_cost[i] + freight_cost[i][j]) * y[i, j] for i in countries for j in countries),
    GRB.MINIMIZE
)

In [175]:
# Constraint: Meet the demand of each market
for j in countries:
    model.addConstr(gp.quicksum(y[i, j] for i in countries) >= demand[j], f"demand_{j}")

# Constraint: Do not exceed the capacity of open plants in each country
for i in countries:
    model.addConstr(
        gp.quicksum(y[i, j] for j in countries) <= (x_low[i] * capacity_low[i] + x_high[i] * capacity_high[i]),
        f"capacity_{i}"
    )

In [176]:
# Solve the optimization problem
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 35 columns and 60 nonzeros
Model fingerprint: 0xdf633cee
Variable types: 25 continuous, 10 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+06]
  Objective range  [5e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+04, 3e+06]
Presolve time: 0.00s
Presolved: 10 rows, 35 columns, 60 nonzeros
Variable types: 25 continuous, 10 integer (0 binary)
Found heuristic solution: objective 5.396050e+07
Found heuristic solution: objective 5.386472e+07

Root relaxation: objective 5.375927e+07, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  

In [177]:
# Output results
if model.status == GRB.OPTIMAL:
    print("✅ Optimal solution found!")

    for c in countries:
        print(f"🌍 {c}: Low Capacity = {x_low[c].x:.0f}, High Capacity = {x_high[c].x:.0f}")

    for i in countries:
        for j in countries:
            if y[i, j].x > 0:
                if i == j:
                    print(f"🏭 {i}: No sourcing, {y[i, j].x:.0f} units remain internal")
                else:
                    print(f"🚢 Ship {y[i, j].x:.0f} units from {i} to {j}")
else:
    print("❌ No optimal solution found.")

✅ Optimal solution found!
🌍 USA: Low Capacity = -0, High Capacity = 2
🌍 Germany: Low Capacity = 1, High Capacity = -0
🌍 Japan: Low Capacity = 1, High Capacity = 1
🌍 Brazil: Low Capacity = 1, High Capacity = -0
🌍 India: Low Capacity = 1, High Capacity = -0
🏭 USA: No sourcing, 2800000 units remain internal
🏭 Germany: No sourcing, 90000 units remain internal
🏭 Japan: No sourcing, 1700000 units remain internal
🏭 Brazil: No sourcing, 145000 units remain internal
🏭 India: No sourcing, 160000 units remain internal


In [183]:
total_variable_cost = 0
total_freight_cost = 0
total_fixed_cost = sum(fixed_cost_low[i] * x_low[i].x + fixed_cost_high[i] * x_high[i].x for i in countries)

for i in countries:
    for j in countries:
        if y[i, j].x > 0:
            shipment_units = y[i, j].x
            shipment_variable_cost = variable_cost[i] * shipment_units
            total_variable_cost += shipment_variable_cost
            if i == j:
                print(f"🏭 {i}: No outsourcing, {shipment_units:.0f} units remain internal")
                print(f"   🔹 Variable Cost: {shipment_variable_cost:.2f}")
            else:
                shipment_freight_cost = freight_cost[i][j] * shipment_units
                total_freight_cost += shipment_freight_cost

                print(f"🚢 Shipment {i} → {j}: {shipment_units:.0f} units")
                print(f"   🔹 Variable Cost: {shipment_variable_cost:.2f}")
                print(f"   🚚 Freight Cost: {shipment_freight_cost:.2f}")

total_shipment_cost = total_variable_cost + total_freight_cost
total_cost = total_fixed_cost + total_shipment_cost

print(f"\n💰 Total Fixed Cost: {total_fixed_cost:.0f}")
print(f"🔹 Total Variable Cost: {total_variable_cost:.0f}")
print(f"🚚 Total Freight Cost: {total_freight_cost:.0f}")
print(f"✅ Final Total Cost: {total_cost:.0f}")

🏭 USA: No outsourcing, 2800000 units remain internal
   🔹 Variable Cost: 33600000.00
🏭 Germany: No outsourcing, 90000 units remain internal
   🔹 Variable Cost: 1170000.00
🏭 Japan: No outsourcing, 1700000 units remain internal
   🔹 Variable Cost: 17000000.00
🏭 Brazil: No outsourcing, 145000 units remain internal
   🔹 Variable Cost: 1160000.00
🏭 India: No outsourcing, 160000 units remain internal
   🔹 Variable Cost: 800000.00

💰 Total Fixed Cost: 44650
🔹 Total Variable Cost: 53730000
🚚 Total Freight Cost: 0
✅ Final Total Cost: 53774650
